<a href="https://colab.research.google.com/github/Matheus0820/Introducao-a-Otimizacao/blob/main/Problema_de_forma%C3%A7%C3%A3o_de_equipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problema da formação de equipe em esporte aquatico
## Descrição do problema:
- Foi solicitado a criação de um algoritimo, no qual entre os 10 melhores jogadores forme uma equipe que minimize o tempo de prova

## Paramêtros:
- $n$: Quantidade de atletas
- $m$: Quantidade de estilos de prova
- $t_{ij}$: Tempo de execução do atleta $i$ na prova $j$

## Variáveis:
- $x_{ij}$: $\begin{cases}
      \mbox{Atleta i recebe prova j}, & x = 1 \\
      \mbox{Atleta i não recebe proja j}, & x = 0
  \end{cases}$

- $y_i$: $\begin{cases}
      \mbox{Atleta i participa da equipe}, & x = 1 \\
      \mbox{Atleta i não participa da equipe}, & x = 0
  \end{cases}$

## Função objetivo:

$$
\mbox{min Z} = \sum_{i = 1}^{n}\sum_{j=1}^{m} t_{ij}x_{ij} \tag{1}
$$

# Sujeito a:
$$
\sum_{i = 1}^{n}x_{ij}y_i = 1, ∀ j = 1,..,m \tag{2}
$$

$$
\sum_{j = 1}^{m}x_{ij} ≤ y_i, ∀ i = 1,..,n \tag{3}
$$

$$
\sum_{i = 1}^{n}y_i = m \tag{4}
$$

# Baixando pedências do AMPL

In [ ]:
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook
ampl = ampl_notebook(
    modules=["highs", "cbc", "gurobi", "cplex"], # pick from over 20 modules including most commercial and open-source solvers
    license_uuid="79c45049-0fee-47f7-a329-ec026ff58c82") # your license UUID (e.g., free ampl.com/ce or ampl.com/courses licenses)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 36.0 MB/s eta 0:00:00
Licensed to AMPL Academic Community Edition License for <matheus.ramos.703@ufrn.edu.br>.


# Pegando dados e gerando arquivo dados.dat

In [ ]:
import json
import numpy as np

categorias = ['borboleta', 'costa', 'livre', 'peito']
#atletas_tempo = np.zeros(40, 5)

# Exportando dados de arquivo
def exportar_dados(categorias):
  lista_atletas = []
  lista_tempo = []
  for i in range(len(categorias)):
    with open(f'ranking_{categorias[i]}.json', 'r') as f:
      dados = json.load(f)['items'][:10]

    for j in range(len(dados)):
      lista_atletas.append(f'"{dados[j]["nome_evento"]}"')
      if categorias[i] == 'borboleta':
        lista_tempo.append([dados[j]["tempo"], 100000, 100000, 100000])
      elif categorias[i] == 'costa':
        lista_tempo.append([100000, dados[j]["tempo"], 100000, 100000])
      elif categorias[i] == 'livre':
        lista_tempo.append([100000, 100000, dados[j]["tempo"], 100000])
      else:
        lista_tempo.append([100000, 100000, 100000, dados[j]["tempo"]])

  return lista_atletas, lista_tempo

# Formatando dados para tabela do AMPL
atletas, t_ij = exportar_dados(categorias)

atletas = f"param atletas := \n" + "\n".join([f"{i+1} {atletas[i]}" for i in range(len(atletas))]) + ";"
estilo = f"param estilo := \n" + "\n".join([f"{i+1} {categorias[i]}" for i in range(len(categorias))]) + ";"
t_ij = f"param t : 1 2 3 4 := \n" + "\n".join(f"{i+1} {t_ij[i][0]} {t_ij[i][1]} {t_ij[i][2]} {t_ij[i][3]}" for i in range(len(t_ij))) + ";"

dados_gerais = "param n := 40; \nparam m := 4;" + "\n" + atletas + "\n" + estilo + "\n" + t_ij

# Colocando dados dentro do arquivo dados.dat
with open('dados.dat', 'w') as f:
  f.write(dados_gerais)
  print("Arquivo dados.dat gerado com sucesso!")


Arquivo dados.dat gerado com sucesso!


# Arquivo modelo.mod

In [ ]:
%%writefile modelo.mod
param n >= 1;
param m >= 1;
param atletas {1..n} symbolic;
param estilo {1..m} symbolic;
param t {1..n, 1..m};

var x {1..n, 1..m} binary;
var y {1..n} binary;

minimize Z: sum{i in 1..n, j in 1..m} t[i,j]*x[i,j];

subject to rest1 {j in 1..m}: sum{i in 1..n} x[i, j] * y[i] == 1;
subject to rest2 {i in 1..n}: sum{j in 1..m} x[i, j] <= y[i];
subject to rest3: sum{i in 1..n} y[i] == m;

Overwriting modelo.mod


# Configurando e execultando algoritimo
> Usando o: **%%ampl_eval**

In [ ]:
%%ampl_eval
reset;
model modelo.mod;
data dados.dat;
option solver cplex;
solve;
display Z;

printf "Atletas selecionados: \n";
for{i in 1..n, j in 1..m} {
    if x[i, j] == 1 then {
        printf "- %s - Estilo: %s\n", atletas[i], estilo[j];
    }
}



CPLEX 22.1.2: CPLEX 22.1.2: optimal solution; objective 21888
4 simplex iterations
Z = 21888

Atletas selecionados: 
- ARTHUR ALMEIDA PINHEIRO - Estilo: borboleta
- GUILHERME AUGUSTO PAGUNG - Estilo: costa
- JOÃO NUNES NETO - Estilo: livre
- KAUÃ SANTOS CARVALHO - Estilo: peito
